# Computing NDVI

In [1]:
import astraea.spark.rasterframes._
import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import geotrellis.spark._
import geotrellis.spark.io._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

implicit val spark = SparkSession.builder().
  master("local[*]").appName("RasterFrames").getOrCreate().withRasterFrames
spark.sparkContext.setLogLevel("ERROR")
import spark.implicits._

val scene = SinglebandGeoTiff("../samples/L8-B8-Robinson-IL.tiff")
val rf = scene.projectedRaster.toRF(128, 128).cache()

Intitializing Scala interpreter ...

Spark Web UI available at http://172.18.0.2:4040
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1531142977598)
SparkSession available as 'spark'


import astraea.spark.rasterframes._
import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import geotrellis.spark._
import geotrellis.spark.io._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@66d037a9
import spark.implicits._
scene: geotrellis.raster.io.geotiff.SinglebandGeoTiff = SinglebandGeoTiff(geotrellis.raster.UShortConstantNoDataArrayTile@6f3e4295,Extent(431902.5, 4313647.5, 443512.5, 4321147.5),EPSG:32616,Tags(Map(AREA_OR_POINT -> POINT),List(Map())),GeoTiffOptions(geotrellis.raster.io.geotiff.Striped@1c1f469c,geotrellis.raster.io.geotiff.compression.DeflateCompression$@1a88560,1,None))
rf: org.apache.spark.sql.DataFrame...

Here's an example of computing the Normalized Differential Vegetation Index (NDVI) is a 
standardized vegetation index which allows us to generate an image highlighting differences in
relative biomass. 

> “An NDVI is often used worldwide to monitor drought, monitor and predict agricultural production, assist in predicting hazardous fire zones, and map desert encroachment. The NDVI is preferred for global vegetation monitoring because it helps to compensate for changing illumination conditions, surface slope, aspect, and other extraneous factors” (Lillesand. *Remote sensing and image interpretation*. 2004).

In [2]:
def redBand = SinglebandGeoTiff("../samples/L8-B4-Elkton-VA.tiff").projectedRaster.toRF("red_band")
def nirBand = SinglebandGeoTiff("../samples/L8-B5-Elkton-VA.tiff").projectedRaster.toRF("nir_band")

// Define UDF for computing NDVI from red and NIR bands
val ndvi = udf((red: Tile, nir: Tile) ⇒ {
  val redd = red.convert(DoubleConstantNoDataCellType)
  val nird = nir.convert(DoubleConstantNoDataCellType)
  (nird - redd)/(nird + redd)
})

// We use `asRF` to indicate we know the structure still conforms to RasterFrame constraints
val rf = redBand.spatialJoin(nirBand).withColumn("ndvi", ndvi($"red_band", $"nir_band")).asRF

redBand: astraea.spark.rasterframes.RasterFrame
nirBand: astraea.spark.rasterframes.RasterFrame
ndvi: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function2>,org.apache.spark.sql.gt.types.TileUDT@29b43b46,Some(List(org.apache.spark.sql.gt.types.TileUDT@29b43b46, org.apache.spark.sql.gt.types.TileUDT@29b43b46)))
rf: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, red_band: rf_tile ... 2 more fields]


In [4]:
rf.select(tileStats($"ndvi")).show()

+---------+--------------------+------------------+-------------------+--------------------+
|dataCells|                 min|               max|               mean|            variance|
+---------+--------------------+------------------+-------------------+--------------------+
|    31433|-0.12488999119929595|0.6699834571985877|0.45558948033745933|0.010907294541915387|
+---------+--------------------+------------------+-------------------+--------------------+



In [3]:
val pr = rf.toRaster($"ndvi", 466, 428)

val brownToGreen = ColorRamp(
  RGBA(166,97,26,255),
  RGBA(223,194,125,255),
  RGBA(245,245,245,255),
  RGBA(128,205,193,255),
  RGBA(1,133,113,255)
).stops(128)

val colors = ColorMap.fromQuantileBreaks(pr.tile.histogramDouble(), brownToGreen)
// change writing location
pr.tile.color(colors).renderPng().write("target/scala-2.11/tut/apps/rf-ndvi.png")

//For a georefrenced singleband greyscale image, could do: `GeoTiff(pr).write("ndvi.tiff")`

spark.stop()

java.io.FileNotFoundException:  target/scala-2.11/tut/apps/rf-ndvi.png (No such file or directory)